In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam

# Read training and test data
test = pd.read_csv("/content/Transfer data project - 2024 WRs (4).csv")
train = pd.read_csv("/content/Transfer data project - SEC Transfer WRS Before (4).csv")

test = test.drop(columns = ["Name", "Year", "Former School", "PBLK(Grade)", "Penalties"])
train = train.drop(columns = ["Name", "Year", "Prev School", "PBLK(Grade)", "Penalties"])


# Correlation matrix for initial data exploration
print(train.corr())

# Clearing environment
del train, test

# Load Keras
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

# Read training data again
train = pd.read_csv("/content/Transfer data project - SEC Transfer WRS Before (4).csv")


# Scaling data
indices_to_scale = [3,4,5,6,7,8,9,10,11,12,13,14,15,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,44]
indices_to_scale_test = [1,2,3,4,5,6,7,8,9,10,11]
#print(indices_to_scale)
scaler = MinMaxScaler()
train_scaled = train.copy()  # Make a copy of the original DataFrame
train_scaled.iloc[:, indices_to_scale] = scaler.fit_transform(train.iloc[:, indices_to_scale])


X = train_scaled.iloc[:, indices_to_scale]

y = train_scaled.iloc[:, [45]]

#x = train_scaled.iloc[:, indices_to_scale_test]

print(y.shape)
print(X.shape)
# Creating Keras model
model = Sequential()
model.add(Dense(units=64, activation='relu', input_dim=X.shape[1]))
model.add(Dense(units=32, activation='relu'))
model.add(Dense(units=y.shape[1], activation='linear'))  # Adjusted to match the number of columns in y

# Compiling model
model.compile(loss='mae', optimizer='adam', metrics=['accuracy'])

# Splitting into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

# Defining early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Training model
history = model.fit(X_train, y_train,  batch_size=64, epochs=500, validation_split=0.1, callbacks=[early_stopping])

# Store number of epochs for future reference
num_epochs_run = len(history.history['val_loss'])

# Testing model
model.evaluate(X_test, y_test)

# Creating Keras model
final_model = Sequential()
final_model.add(Dense(units=64, activation='relu', input_dim=X.shape[1]))
final_model.add(Dense(units=32, activation='relu'))
final_model.add(Dense(units=y.shape[1], activation='linear'))  # Adjusted to match the number of columns in y

# Compiling model
final_model.compile(loss='mae', optimizer='adam', metrics=['accuracy'])

# Training final model
final_model.fit(X, y, epochs=int(num_epochs_run*1.15), batch_size=32)

# Read actual testing data
final_test = pd.read_csv("/content/Transfer data project - 2024 WRs (4).csv")


# Scale testing data
final_test_scaled = scaler.transform(final_test.iloc[:, indices_to_scale])

# Predict new activity ids using model
print(final_test_scaled.shape)
print(model.input_shape)
predictions = final_model.predict(final_test_scaled)
#for p in predictions:
  #fp = p.replace(" ", ",")
  #print(fp)
#p2d = predictions.reshape(-1, 2)
#print(p2d)
string_list = [' '.join(map(str, sublist)) for sublist in predictions]

# Join the strings with newline character '\n' to separate them
final_string = '\n'.join(string_list)


print(predictions)

# Assign values to new dataframe
final_test['Predictions1'] = (string_list)
#final_test['Predictions2'] = np.nonzero(predictions)

# Print the updated DataFrame
print(final_test)

# Exporting dataset
final_test.to_csv("ACC_test_with_predictions.csv", index=False)

                        Conference    Height    Weight       Tgt       Rec  \
Conference                1.000000 -0.431859 -0.293086  0.037684  0.045610   
Height                   -0.431859  1.000000  0.691509 -0.259134 -0.308766   
Weight                   -0.293086  0.691509  1.000000 -0.141399 -0.183518   
Tgt                       0.037684 -0.259134 -0.141399  1.000000  0.973613   
Rec                       0.045610 -0.308766 -0.183518  0.973613  1.000000   
Rec%                      0.097910 -0.336257 -0.284447  0.257648  0.394980   
Yds                      -0.045530 -0.244504 -0.139225  0.945851  0.927332   
Yd/rec                   -0.152215  0.009258 -0.129605 -0.120618 -0.192980   
TD                       -0.181834  0.003082  0.027987  0.819303  0.746868   
OFF(Grade)               -0.178202 -0.299262 -0.195790  0.661175  0.693313   
RECV(Grade)              -0.176990 -0.312759 -0.204155  0.668164  0.699123   
DROP(Grade)              -0.040618 -0.360133 -0.287850  0.319552

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.0000e+00 - loss: 75.2128 - val_accuracy: 0.0000e+00 - val_loss: 73.0363
Epoch 2/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - accuracy: 0.0000e+00 - loss: 75.1263 - val_accuracy: 0.0000e+00 - val_loss: 72.9400
Epoch 3/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - accuracy: 0.0000e+00 - loss: 75.0375 - val_accuracy: 0.0000e+00 - val_loss: 72.8416
Epoch 4/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - accuracy: 0.0000e+00 - loss: 74.9461 - val_accuracy: 0.0000e+00 - val_loss: 72.7409
Epoch 5/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 0.0000e+00 - loss: 74.8512 - val_accuracy: 0.0000e+00 - val_loss: 72.6402
Epoch 6/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 0.0000e+00 - loss: 74.7532 - val_accuracy: 0.0000e+00 - val_loss: 72.5385
Epoch 7/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 0.0000e+00 - loss: 74.6520 - val_accuracy: 0.0000e+00 - val_loss: 72.4347
Epoch 8/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - accuracy: 

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.0000e+00 - loss: 74.6183
Epoch 2/104
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.0000e+00 - loss: 74.2543 
Epoch 3/104
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.0000e+00 - loss: 74.6747 
Epoch 4/104
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.0000e+00 - loss: 73.6676 
Epoch 5/104
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.0000e+00 - loss: 73.7891 
Epoch 6/104
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.0000e+00 - loss: 73.4194 
Epoch 7/104
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.0000e+00 - loss: 72.5988 
Epoch 8/104
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.0000e+00 - loss: 72.5821 
Epoch 9/104
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.0000e+00 - loss: 72.6730 
Epoch 10/104
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.0000e+00 - loss: 71.6739 
Epoch 11/104
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.0000e+00 - loss: 71.4105 
Epoch 12/104
2/2 ━━━━━━━━━━━━━━━

In [ ]:
from google.colab import drive
drive.mount('/content/drive')